## We Merge different Sources into Single One

Starting from:
1. Fantacalcio csv
2. Gazzetta, Corriere e Tuttosport csv
3. WyScout Dataframe

Are computed **match** between names in order to retrive a specific id for the player (wyscout id)

A **final DF** contained name marks and wy_id is given in output

In [1]:
#All python packages
import pandas as pd
import numpy as np
from datetime import datetime
import pprint
from collections import OrderedDict 
from operator import getitem 
import textdistance
import os

In [2]:
DATA_PATH = '../data'
#Import Dataframe
datasetFantacalcio = pd.read_csv(os.path.join(DATA_PATH,'fantacalcioScore_2017_2018.csv'))
datasetOther = pd.read_csv(os.path.join(DATA_PATH, 'alltheotherScore_2017_2018.csv'))
datasetWyScout = pd.read_csv(os.path.join(DATA_PATH,'wy_scout_player_association_name_id.csv'))

### Core Function
#### Only Exact Match
Take a dataset as parameter and after matching the names assign to each name a specific wyscoutid

In [3]:
#this function try to assign a wyscoutid to dataset
def createMatch(dataset, datasetWyScout1):
    """
    Compute a check for each record in param1 (dataset) in order to find out an exact match with some player in name in param2
    (datasetWyScout1)
    
    Parameters
    ----------
    DF : dataset -> pandas dataframe from fantacalcio marks or ts cr and gazzetta marks
    DF : datasetWyScout1 - > pandas dataframe from wy scout data
    
    Returns
    -------
    Dict
        Dict of exact match, structured as player name and wy id 
    """
    startTime = datetime.now()
    print('Starting Names Matching...')
    print(startTime)
    
    #list of id to return
    #idRederences = []
    #dictionaries where we store the couple name id
    alreadyMatched = {}
    
    pprint.pprint(len(dataset))
    
    #for each player in input dataset
    for player in dataset.values:
        possibleChoices = set()
        #check if we already assigned something to the player
        if(player[2] in alreadyMatched):
            #idRederences.append(alreadyMatched[player[2]])
            dosomething = ''
        else:
            #for each player in wyscoutdataset
            for wyPlayer in datasetWyScout1.values:
                #same team same role
                if(player[0] == wyPlayer[6] and player[3] == wyPlayer[5]):
                    #exactly the same name
                    if(player[2] == wyPlayer[2]):
                        #save the reference
                        #idRederences.append(wyPlayer[1])
                        alreadyMatched[player[2]] = wyPlayer[1]
                        break
                    #last name is inside the dataset name, could be a possible match
                    if(wyPlayer[4] in player[2] or wyPlayer[4] == player[2]):
                        #insert references inside a possible choiche
                        possibleChoices.add((wyPlayer[2],wyPlayer[1]))
                    #first name is inside the dataset name, could be a possible match
                    if(wyPlayer[3] in player[2] or wyPlayer[3] == player[2]):
                        #insert references inside a possible choiche
                        possibleChoices.add((wyPlayer[2],wyPlayer[1]))
        #we retrive only one name
        if(len(possibleChoices) == 1):
            for el in possibleChoices:
                #idRederences.append(el[1])
                alreadyMatched[player[2]] = el[1]
                break
    
    pprint.pprint('Size of Players Well defined '+ str(len(alreadyMatched))+'')
    
    print('End Names Matching...')
    print(datetime.now() - startTime)  

    return alreadyMatched
            

In [4]:
playerFantacalcioMatch = createMatch(datasetFantacalcio, datasetWyScout)
playerOtherMatch = createMatch(datasetOther, datasetWyScout)

Starting Names Matching...
2020-06-19 11:12:24.482865
10588
'Size of Players Well defined 215'
End Names Matching...
0:00:03.269602
Starting Names Matching...
2020-06-19 11:12:27.754199
9788
'Size of Players Well defined 231'
End Names Matching...
0:00:02.581733


### N.B. Keep mantain the dictionary from this output because we will use it later

We look for the number of instances that were captured with exact or partial match similarity

In [5]:
#The function need as parameter a dataset of player and a dictionary (composed by key:name of the player and wyid as value)
def createIdArray(datasetFantacalcio1, playerFantacalcioMatch1):
    """
    Create an array of ids starting from a df of player and a dictionary of playernames and wy id
    
    Parameters
    ----------
    DF : datasetFantacalcio1 -> pandas df that contain player and marks without wy id
    DICT : playerFantacalcioMatch1 -> dictionary that contains as a key the player name and as value the player wy id
        List of player names.
    
    Returns
    -------
    List
        List id ordered as the pandas df in input.
    """
    wyIdListRef = []
    #for each player in the dataset
    for playerFantacalcio in datasetFantacalcio1.values:
        #check if the player is matched
        if(playerFantacalcio[2] in playerFantacalcioMatch1):
            #if matched append the wyid
            wyIdListRef.append(playerFantacalcioMatch1[playerFantacalcio[2]])
        else:
            #if not matched append -1
            wyIdListRef.append(-1) 
    return wyIdListRef        

In [6]:
wyIdListRefFanta = createIdArray(datasetFantacalcio, playerFantacalcioMatch)
wyIdListRefOther = createIdArray(datasetOther, playerOtherMatch)

In [7]:
datasetFantacalcio['wy_IDS'] = wyIdListRefFanta
datasetOther['wy_IDS'] = wyIdListRefOther

extract those record that are not matched with no one id

In [8]:
dfRemainedFantacalcio = datasetFantacalcio[datasetFantacalcio['wy_IDS'] == -1]
dfRemainedOther = datasetOther[datasetOther['wy_IDS'] == -1]

#### Edit distance

This distance is computed by finding the number of edits which will transform one string to another. The transformations allowed are:
* insertion — adding a new character,
* deletion — deleting a character,
* substitution — replace one character by another.

In [9]:
def levenshtein_ratio_and_distance(s, t, ratio_calc = False):
    """
    Compute the edit distance between two given strings
    
    Parameters
    ----------
    String : s
    String : t
    Boolean: ratio_calc -> infer if the ratio calculation is needed
    
    Returns
    -------
    Float
        Ratio of edit distance similarity.
    """
    # Initialize matrix of zeros
    rows = len(s)+1
    cols = len(t)+1
    distance = np.zeros((rows,cols),dtype = int)

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions    
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                # In order to align the results with those of the Python Levenshtein package, if we choose to calculate the ratio
                # the cost of a substitution is 2. If we calculate just distance, then the cost of a substitution is 1.
                if ratio_calc == True:
                    cost = 2
                else:
                    cost = 1
            distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                 distance[row][col-1] + 1,          # Cost of insertions
                                 distance[row-1][col-1] + cost)     # Cost of substitutions
    if ratio_calc == True:
        # Computation of the Levenshtein Distance Ratio
        Ratio = ((len(s)+len(t)) - distance[row][col]) / (len(s)+len(t))
        return Ratio
    else:
        print(distance)
        # This is the minimum number of edits needed to convert string a to string b
        return "The strings are {} edits away".format(distance[row][col])

In [10]:
Distance = levenshtein_ratio_and_distance('LEIVA L', 'LUCAS LEIVA')
print(Distance)
Ratio = levenshtein_ratio_and_distance('LEIVA L', 'LUCAS LEIVA',ratio_calc = True)
print(Ratio)

[[ 0  1  2  3  4  5  6  7  8  9 10 11]
 [ 1  0  1  2  3  4  5  6  7  8  9 10]
 [ 2  1  1  2  3  4  5  6  6  7  8  9]
 [ 3  2  2  2  3  4  5  6  7  6  7  8]
 [ 4  3  3  3  3  4  5  6  7  7  6  7]
 [ 5  4  4  4  3  4  5  6  7  8  7  6]
 [ 6  5  5  5  4  4  4  5  6  7  8  7]
 [ 7  6  6  6  5  5  5  4  5  6  7  8]]
The strings are 8 edits away
0.5555555555555556


#### Populate those player that doesn't have ID

In [11]:
#This function take as input the df with the remaining player without id and take the wyscout df
def populateRemaining(df, wyS, alreadyMatchedPlayer):
    """
    Compute a check for each record in param1 (df) in order to find out an similar match (based on edit distance)
    with some player in name in param2 (wyS). Note that the dict alreadyMatchedPlayer help in speed up the computation
    
    Parameters
    ----------
    DF : df -> pandas dataframe from fantacalcio marks or ts cr and gazzetta marks
    DF : wyS - > pandas dataframe from wy scout data
    Dict : alreadyMatchedPlayer -> dictionary of player already matched with some id 
    
    Returns
    -------
    Dict
        Dict of exact match, structured as player name and wy id 
    """
    startTime = datetime.now()
    print('Starting Names Matching...')
    print(startTime)
    
    #list of id to return
    #idRederences = []
    #dictionaries where we store the couple name id
    alreadyMatched = {}
    
    pprint.pprint(len(df))
    
    #for each player in input dataset
    for player in df.values:
        possibleChoices = set()
        #check if we already assigned something to the player
        if(player[2] in alreadyMatched):
            dosomething = ''
        else:
            #for each player in wyscoutdataset
            for wyPlayer in wyS.values:
                #same team and not matched
                if(player[0] == wyPlayer[6] and not(wyPlayer[1] in alreadyMatchedPlayer.values())):
                    #we don't assign a possible choice if the player role is P and the wyscout role is another
                    if(player[3] == 'P' and not(wyPlayer[5] == 'P')):
                        donoting = ''
                    elif(not(player[3] == 'P') and wyPlayer[5] == 'P'):
                        donoting = ''
                    else:
                        possibleChoices.add((wyPlayer[2],wyPlayer[1]))
        # we discover more matches inside the data
        if(len(possibleChoices) > 1):
            listOfPossibilities = []
            #for each possible match
            for el in possibleChoices:
                possibleMatch = {}
                #we compute the levenshtein distance for each name
                RatioLev = levenshtein_ratio_and_distance(player[2],el[0],ratio_calc = True)
                #we compute the Ratcliff-Obershelp similarity
                RatioRat = textdistance.ratcliff_obershelp(player[2],el[0])
                #save name
                possibleMatch['Name'] = el[0]
                #save ratio for lev
                possibleMatch['Ratio Lev'] = RatioLev
                #save ratio for ratcliff
                possibleMatch['Ratio Rat'] = RatioRat
                #save id
                possibleMatch['ID'] = el[1]
                
                listOfPossibilities.append(possibleMatch)
            #we iterate all the possible choiches, so we go for the closest
            closestPlayer = sortListOfDict(listOfPossibilities)
            #similarityTreshold
            if(closestPlayer[0]['Ratio Lev'] > 0.5 or closestPlayer[0]['Ratio Rat'] > 0.5):
                idToAssign = closestPlayer[0]['ID']
                alreadyMatched[player[2]] = idToAssign 
        
    
    pprint.pprint('Size of Players Well defined '+ str(len(alreadyMatched))+'')
    
    print('End Names Matching...')
    print(datetime.now() - startTime)  

    return alreadyMatched

In [12]:
#sort a list of dictionary in DESCENDING mode
def sortListOfDict(lis):
    return sorted(lis, key = lambda i: i['Ratio Lev'],reverse=True) 

In [13]:
remainingFantacalcio = populateRemaining(dfRemainedFantacalcio, datasetWyScout, playerFantacalcioMatch)
remainingOther = populateRemaining(dfRemainedOther, datasetWyScout, playerOtherMatch)

Starting Names Matching...
2020-06-19 11:13:03.739685
5482
'Size of Players Well defined 77'
End Names Matching...
0:00:31.863650
Starting Names Matching...
2020-06-19 11:13:35.603679
4887
'Size of Players Well defined 122'
End Names Matching...
0:00:24.736915


In [14]:
fantacalcioMatchings = {**playerFantacalcioMatch, **remainingFantacalcio}
otherMatchings = {**playerOtherMatch, **remainingOther}

In [15]:
wyIdListRefFantacalcio = createIdArray(datasetFantacalcio, fantacalcioMatchings)
datasetFantacalcio['wy_IDS'] = wyIdListRefFantacalcio

wyIdListRefOther = createIdArray(datasetOther, otherMatchings)
datasetOther['wy_IDS'] = wyIdListRefOther

In [16]:
manualFanta = datasetFantacalcio[datasetFantacalcio['wy_IDS'] == -1]
manualOther = datasetOther[datasetOther['wy_IDS'] == -1]

In [17]:
manualFanta = manualFanta.groupby(['team', 'player'])
manualOther = manualOther.groupby(['team', 'player'])

In [18]:
result1 = manualFanta.first()

In [19]:
result2 = manualOther.first()

# Some player aren't inside the wyscout df
we extract them in order to retrive the names and team

We need to make some reasons over it

#### HERE WE WRITE SOME CODE TO EXTRACT  THESE INFORMATION

~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 

In [20]:
#this function check if the df we insert as first parameter has some name that are already paired to an id
#return the list of index to remove
def removeAlreadyFind(df_merge_col, datasetOther):
    """
    Check if the df we insert as first parameter has some name that are already paired to an id
    
    Parameters
    ----------
    DF : df_merge_col -> pandas dataframe from reamaining record (no id) fantacalcio marks or ts cr and gazzetta marks
    DF : datasetOther - > pandas dataframe from fantacalcio marks or ts cr and gazzetta marks
    
    Returns
    -------
    List
        List of index to remove (player that have already an id found)
    """
    listindex = []
    index = 0
    for el in df_merge_col.values:
        drop = False
        for el1 in datasetOther.values:
            #same name
            if(el1[0] == el[0] and el[1] == el1[2] and el1[len(datasetOther.columns) - 1] != -1):
                drop = True
        if(drop):
            listindex.append(index)
        index = index + 1
    return listindex

#### we start the last phase
*Remove already paired
*Final check the one that has no match with all the other inside the all player df (no restriction about team and role) -> this is done due to the absence of team informations for some player





In [21]:
dfplayer = pd.read_json(os.path.join(DATA_PATH, 'players_new.json'),encoding = "unicode-escape")
dfplayer = dfplayer.reindex(sorted(dfplayer.columns), axis=1)
listFirst = []
listId = []
listIdTeam = []
listLast = []
listRole = []
listShort = []
for el in dfplayer.values:
    listFirst.append(el[4].upper())
    listId.append(el[16])
    listIdTeam.append(el[3])
    listLast.append(el[9].upper())
    listRole.append(el[12]['code3'])
    listShort.append(el[13].upper())
newRole = []
for role in listRole:
    if(role == 'GKP'):
        newRole.append('P')
    if(role == 'DEF'):
        newRole.append('D')
    if(role == 'MID'):
        newRole.append('C')
    if(role == 'FWD'):
        newRole.append('A')
listRole = newRole

newAllPlayer = pd.DataFrame({'first' : listFirst, 'id_player' : listId, 'id_team': listIdTeam, 'last': listLast, 'role': listRole, 'short': listShort})
newAllPlayer

,first,id_player,id_team,last,role,short
0,ANDREJ,385025,11954.0,KOTNIK,C,A. KOTNIK
1,GIOVANNI,368642,3435.0,CROCIATA,C,G. CROCIATA
2,ANTONIO,20483,3158.0,MIRANTE,P,A. MIRANTE
3,STEVEN,8200,3158.0,N'ZONZI,C,S. N'ZONZI
4,GABRIEL ALEJANDRO,20490,9142.0,PALETTA,D,G. PALETTA
...,...,...,...,...,...,...
1059,SIMONE,20472,3173.0,PADOIN,C,S. PADOIN
1060,ROGÉRIO,403449,3315.0,OLIVEIRA DA SILVA,D,ROGÉRIO
1061,MARCO,20476,NaN,BORRIELLO,A,M. BORRIELLO
1062,ALESSANDRO,20478,3315.0,MATRI,A,A. MATRI


Mine for each result the respective list of names not paired

In [22]:
listzzzz = []
listpep = []
for el in result1.index:
    listzzzz.append(el[0])
    listpep.append(el[1])

In [23]:
toFillFanta = pd.DataFrame({'team': listzzzz, 'player': listpep})

In [24]:
toFillFanta

,team,player
0,ATALANTA,BASTONI
1,ATALANTA,CALDARA
2,ATALANTA,CORNELIUS
3,ATALANTA,HAAS
4,ATALANTA,PETAGNA
...,...,...
240,VERONA,VALOTI
241,VERONA,VERDE
242,VERONA,VUKOVIC
243,VERONA,ZUCULINI B


In [25]:
listzzzz = []
listpep = []
for el in result2.index:
    listzzzz.append(el[0])
    listpep.append(el[1])

In [26]:
toFillOther = pd.DataFrame({'team': listzzzz, 'player': listpep})

In [27]:
toFillOther

,team,player
0,ATALANTA,BASTONI A
1,ATALANTA,CALDARA M
2,ATALANTA,CORNELIUS A
3,ATALANTA,CRISTANTE B
4,ATALANTA,HAAS N
...,...,...
234,VERONA,SOUPRAYEN S
235,VERONA,VERDE D
236,VERONA,VUKOVIC J
237,VERONA,ZUCULINI B


Remove the player already paired

In [28]:
indexToRemoveFanta = removeAlreadyFind(toFillFanta, datasetFantacalcio)
indexToRemoveOther = removeAlreadyFind(toFillOther, datasetOther)

In [29]:
toFillFanta = toFillFanta.drop(indexToRemoveFanta)
toFillOther = toFillOther.drop(indexToRemoveOther)

#### All player dataset scan
In this section we look for those record that are close to our remaining using the whole dataset of player 
(contry and team are not considered)

In [30]:
#function that take the maximum of 3 values
def maximumof3(RatioLevShort, RatioLevFirst, RatioLevLast):
    """
    Function that takes the maximum between 3 numbers
    
    Parameters
    ----------
    Float : 
    Float : 
    Float : 
    
    Returns
    -------
    Float
        Biggest Number between the one in input.
    """
    
    if (RatioLevShort >= RatioLevFirst) and (RatioLevShort >= RatioLevLast):
        return RatioLevShort
    elif (RatioLevFirst >= RatioLevShort) and (RatioLevFirst >= RatioLevLast):
        return RatioLevFirst
    else:
        return RatioLevLast
   

## N.B. COULD BE SLOW

In [31]:
#for each player remained compute string similarity with all the player in all player dataset
def matchwithallplayerlist(df_merge_col, allPlayers):
    """
    Go deep into all players (without role or team matching) in order to retrive id for some players 
    that doesn't have information about team
    
    Parameters
    ----------
    DF : df_merge_col -> pandas dataframe from remained (no id) fantacalcio marks or ts cr and gazzetta marks
    DF : allplayers - > pandas dataframe from wy scout data that contained all player information
    
    Returns
    -------
    Dict
        Dict of match, structured as player name and wy id 
    """
    
    startTime = datetime.now()
    print('Starting Names Matching...')
    print(startTime)
    
    matches = {}
    for el in df_merge_col.values:
        possiblePlay = []
        for play in allPlayers.values:
            #we compute the levenshtein distance for each name
            RatioLevShort = levenshtein_ratio_and_distance(el[1],play[5],ratio_calc = True)
            #we compute the Ratcliff-Obershelp similarity
            RatioRatShort = textdistance.ratcliff_obershelp(el[1],play[5])
            #we compute the levenshtein distance for each name
            RatioLevFirst = levenshtein_ratio_and_distance(el[1],play[0],ratio_calc = True)
            #we compute the Ratcliff-Obershelp similarity
            RatioRatFirst = textdistance.ratcliff_obershelp(el[1],play[0])
            #we compute the levenshtein distance for each name
            RatioLevLast = levenshtein_ratio_and_distance(el[1],play[3],ratio_calc = True)
            #we compute the Ratcliff-Obershelp similarity
            RatioRatLast = textdistance.ratcliff_obershelp(el[1],play[3])

            bestLev = maximumof3(RatioLevShort, RatioLevFirst, RatioLevLast)

            bestRat = maximumof3(RatioRatShort, RatioRatFirst, RatioRatLast)

            if(bestLev > 0.7 or bestRat > 0.7):
                player = {}
                player['Name'] = play[5]
                player['Ratio Lev'] = bestLev
                player['Ratio Rat'] = bestRat
                player['id'] = int(play[1])
                possiblePlay.append(player)
        matches[el[1]] = possiblePlay
        
    print('End Names Matching...')
    print(datetime.now() - startTime)  
    print('--------------------------------------------------------------------------------------------------')
    return matches

In [32]:
lastCheckFanta = matchwithallplayerlist(toFillFanta, newAllPlayer)
lastCheckOther = matchwithallplayerlist(toFillOther, newAllPlayer)

Starting Names Matching...
2020-06-19 11:14:50.565972
End Names Matching...
0:13:20.680357
--------------------------------------------------------------------------------------------------
Starting Names Matching...
2020-06-19 11:28:11.266203


KeyboardInterrupt: 

#### We look for the closest player

In [38]:
#from a dictionary extract for each element a list
#sort the list and retrive the most important match in terms of rating
def exctractDFFromDictOfList(diz):
    """
    From a dictionary for each element extract a list and than sort that list based on ratio from edit distance
    
    Parameters
    ----------
    Dict : diz - > dictionary that contain for each not matched player a list of possible match in terms of ratio distance
    
    Returns
    -------
    DF
        pandas dataframe with the name and possible match
    """
    fantaName = []
    wyName = []
    wyId = []
    for el in diz:
        listSort = sortListOfDict(diz[el])
        if(len(listSort) > 0):
            fantaName.append(el)
            wyName.append(listSort[0]['Name'])
            wyId.append(listSort[0]['id'])
    return pd.DataFrame({'fanta_name': fantaName, 'wy_name': wyName, 'wy_id': wyId})

In [34]:
filledFanta = exctractDFFromDictOfList(lastCheckFanta)

In [35]:
filledFanta.to_csv(os.path.join(DATA_PATH, 'filledFanta_2017_2018.csv'), index = False)

In [36]:
filledOther = exctractDFFromDictOfList(lastCheckOther)

In [37]:
filledOther.to_csv(os.path.join(DATA_PATH, 'filledOther_2017_2018.csv'), index = False)

## Now Before Continuing

Is important to look on the filled csv, **it could append that some player had a clear mismatch**.
So open them and check by hand is important. 

### Insert -1 where no clear match append 

In [39]:
filledFanta = pd.read_csv(os.path.join(DATA_PATH, 'filledFanta_2017_2018.csv'))

In [40]:
filledOther = pd.read_csv(os.path.join(DATA_PATH, 'filledOther_2017_2018.csv'))

#### Continuing Run These scripts will give as output the last df, making some computation about lenght and merge between dataset

we remove those record that couldn't be identified

In [41]:
filledFanta = filledFanta[filledFanta['wy_id'] != -1]

In [42]:
filledOther = filledOther[filledOther['wy_id'] != -1]

In [43]:
#starting from a dataset this function create a dictionary composed by name and wyid
def manualMatchFromDFtoDict(df):
    match = {}
    for el in df.values:
        match[el[0]] = el[2]
    return match

In [44]:
filledFantaMatching = manualMatchFromDFtoDict(filledFanta)

In [45]:
filledOtherMatching = manualMatchFromDFtoDict(filledOther)

In [46]:
fanta = {**fantacalcioMatchings, **filledFantaMatching}
other = {**otherMatchings, **filledOtherMatching}

In [47]:
fantaRef = createIdArray(datasetFantacalcio, fanta)
otherRef = createIdArray(datasetOther, other)

In [48]:
datasetFantacalcio

,team,match_day,player,position,fantacalcio_score,wy_IDS
0,ATALANTA,1,BERISHA E.,P,6.0,50849
1,ATALANTA,1,CALDARA,D,7.0,-1
2,ATALANTA,1,HATEBOER,D,6.0,280419
3,ATALANTA,1,MASIELLO A.,D,5.5,20841
4,ATALANTA,1,PALOMINO,D,6.0,86366
...,...,...,...,...,...,...
10583,VERONA,38,AARONS,C,5.5,20516
10584,VERONA,38,MATOS,A,5.5,20949
10585,VERONA,38,CERCI,A,6.5,-1
10586,VERONA,38,FARES,A,5.5,-1


In [49]:
datasetFantacalcio['wy_IDS'] = fantaRef
datasetOther['wy_IDS'] = otherRef

In [50]:
datasetOther = datasetOther[datasetOther['wy_IDS'] != -1]

In [51]:
datasetOther.insert(loc=7, column='match_day_other', value=datasetOther['match_day'])
datasetOther

,team,match_day,player,position,gazzetta_score,corriere_score,tuttosport_score,match_day_other,wy_IDS
0,ATALANTA,1,BERISHA E,P,5.5,6.0,6.0,1,50849
1,ATALANTA,1,GOSENS R,D,5.0,5.5,5.0,1,295176
2,ATALANTA,1,HATEBOER H,D,6.0,5.5,6.0,1,280419
3,ATALANTA,1,MASIELLO A,D,5.5,6.0,5.5,1,20841
4,ATALANTA,1,PALOMINO J,D,6.0,6.0,6.0,1,86366
...,...,...,...,...,...,...,...,...,...
9783,VERONA,38,BEARZOTTI E,C,5.0,5.0,4.5,38,388099
9784,VERONA,38,AARONS R,C,5.0,4.5,5.5,38,20516
9785,VERONA,38,DANZI A,C,6.0,6.0,5.0,38,523089
9786,VERONA,38,CERCI A,A,6.5,6.0,6.5,38,21640


In [52]:
datasetFantacalcio

,team,match_day,player,position,fantacalcio_score,wy_IDS
0,ATALANTA,1,BERISHA E.,P,6.0,50849
1,ATALANTA,1,CALDARA,D,7.0,288663
2,ATALANTA,1,HATEBOER,D,6.0,280419
3,ATALANTA,1,MASIELLO A.,D,5.5,20841
4,ATALANTA,1,PALOMINO,D,6.0,86366
...,...,...,...,...,...,...
10583,VERONA,38,AARONS,C,5.5,20516
10584,VERONA,38,MATOS,A,5.5,20949
10585,VERONA,38,CERCI,A,6.5,20947
10586,VERONA,38,FARES,A,5.5,328336


In [53]:
datasetOther.columns = ['team','match_day_other','player','position','gazzetta_score','corriere_score','tuttosport_score','match_day','wy_IDS']

In [56]:
datasetOther.head(0)

,team,match_day_other,player,position,gazzetta_score,corriere_score,tuttosport_score,match_day,wy_IDS


In [57]:
len(datasetOther)

9576

In [58]:
datasetFantacalcio.head(0)

,team,match_day,player,position,fantacalcio_score,wy_IDS


In [59]:
len(datasetFantacalcio)

10588

we join the two dataframe based on wyscout id, team, giornata and position

In [60]:
finalDf = pd.merge(datasetOther, datasetFantacalcio,how='inner', left_on = ['team', 'match_day_other', 'wy_IDS'], right_on = ['team', 'match_day', 'wy_IDS'])

In [61]:
len(finalDf)

6137

In [62]:
finalDf.head(0)

,team,match_day_other,player_x,position_x,gazzetta_score,corriere_score,tuttosport_score,match_day_x,wy_IDS,match_day_y,player_y,position_y,fantacalcio_score


In [63]:
finalDf.columns = ['team', 'match_day', 'player_name_newspaper', 'player_role_newspaper', 'gazzetta_score', 'corriere_score', 'tuttosport_score', 'match_day_newspaper', 'wy_IDS', 'match_day_fanta', 'player_name_fantacalcio', 'player_role_fantacalcio', 'fantacalcio_score']

In [64]:
finalDf

,team,match_day,player_name_newspaper,player_role_newspaper,gazzetta_score,corriere_score,tuttosport_score,match_day_newspaper,wy_IDS,match_day_fanta,player_name_fantacalcio,player_role_fantacalcio,fantacalcio_score
0,ATALANTA,1,BERISHA E,P,5.5,6.0,6.0,1,50849,1,BERISHA E.,P,6.0
1,ATALANTA,1,HATEBOER H,D,6.0,5.5,6.0,1,280419,1,HATEBOER,D,6.0
2,ATALANTA,1,MASIELLO A,D,5.5,6.0,5.5,1,20841,1,MASIELLO A.,D,5.5
3,ATALANTA,1,PALOMINO J,D,6.0,6.0,6.0,1,86366,1,PALOMINO,D,6.0
4,ATALANTA,1,TOLOI R,D,6.0,6.5,5.5,1,41034,1,TOLOI,D,5.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132,VERONA,38,ROMULO O,D,6.5,7.0,6.5,38,20935,38,ROMULO,D,6.5
6133,VERONA,38,SOUPRAYEN S,D,6.0,6.0,6.0,38,26373,38,SOUPRAYEN,D,6.0
6134,VERONA,38,ZUCULINI F,C,6.0,6.0,6.0,38,86309,38,ZUCULINI F,C,6.0
6135,VERONA,38,BEARZOTTI E,C,5.0,5.0,4.5,38,388099,38,BEARZOTTI,D,5.0


In [65]:
finalDf.to_csv(os.path.join(DATA_PATH, 'final_df_season_2017_2018.csv'), index = False)